# Attributes
- Attributes and values for the most part do not work very well for the retrieval. An exception is `Type,generic` dataset which can map to `Subtype` in ILCD.

In [2]:
# Define attribute-schema mapping
attribute_schema_mapping = {
    "EPD_DataSet": [
        "UUID",
        "Version",
        "Name (de)",
        "Name (en)",
        "Kategorie (original)",
        "Kategorie (en)",
        "Konformität",
        "Laenderkennung",
        "Typ",
        "Referenzjahr",
        "Gueltig bis",
        "URL",
        "Declaration owner",
        "Veroeffentlicht am",
        "Registrierungsnummer",
        "Registrierungsstelle",
        "UUID des Vorgängers",
        "Version des Vorgängers",
        "URL des Vorgängers",
        "Modul",
        "Szenario",
        "Szenariobeschreibung",
    ],
    # "EPD_FlowDataSet": [
    #     "PERE",
    #     "PERM",
    #     "PERT",
    #     "PENRE",
    #     "PENRM",
    #     "PENRT",
    #     "SM",
    #     "RSF",
    #     "NRSF",
    #     "FW",
    #     "HWD",
    #     "NHWD",
    #     "RWD",
    #     "CRU",
    #     "MFR",
    #     "MER",
    #     "EEE",
    #     "EET",
    # ],
    # "ILCD_FlowPropertyDataSet": [
    #     "Bezugsgroesse",
    #     "Bezugseinheit",
    #     "Referenzfluss-UUID",
    #     "Referenzfluss-Name",
    #     "Schuettdichte (kg/m3)",
    #     "Flaechengewicht (kg/m2)",
    #     "Rohdichte (kg/m3)",
    #     "Schichtdicke (m)",
    #     "Ergiebigkeit (m2)",
    #     "Laengengewicht (kg/m)",
    #     "Stueckgewicht (kg)",
    #     "Umrechungsfaktor auf 1kg",
    #     "biogener Kohlenstoffgehalt in kg",
    #     "biogener Kohlenstoffgehalt (Verpackung) in kg",
    # ],
    # "ILCD_LCIAMethodDataSet": [
    #     "GWP",
    #     "ODP",
    #     "POCP",
    #     "AP",
    #     "EP",
    #     "ADPE",
    #     "ADPF",
    #     "AP (A2)",
    #     "GWPtotal (A2)",
    #     "GWPbiogenic (A2)",
    #     "GWPfossil (A2)",
    #     "GWPluluc (A2)",
    #     "ETPfw (A2)",
    #     "PM (A2)",
    #     "EPmarine (A2)",
    #     "EPfreshwater (A2)",
    #     "EPterrestrial (A2)",
    #     "HTPc (A2)",
    #     "HTPnc (A2)",
    #     "IRP (A2)",
    #     "SOP (A2)",
    #     "ODP (A2)",
    #     "POCP (A2)",
    #     "ADPF (A2)",
    #     "ADPE (A2)",
    #     "WDP (A2)",
    # ],
}

In [ ]:
# Attributes and Values EPD_DataSet
# attribute_schema_mapping = {
    # "EPD_DataSet": [
    #     'UUID,8ef7b1b9-da21-43c9-a7d1-9a14d9dc7bcf',
    #     'Version,00.01.002',
    #     'Name (de),"Beton C30/37 XC4 XF1 XA1 F3 16 M ECOPact, Rezept Nummer DA5234-DSZK Version 1, Transportbetonwerk Aachen-Haaren, Germany"',
    #     'Name (en),"Beton C30/37 XC4 XF1 XA1 F3 16 M ECOPact, Rezept Nummer DA5234-DSZK Version 1, Transportbetonwerk Aachen-Haaren, Germany"',
    #     "Kategorie (original),'Mineralische Baustoffe' / 'Mörtel und Beton' / 'Beton'",
    #     "Kategorie (en),'Mineral building products' / 'Binder' / 'Cement'",
    #     "Konformität,'EN 15804+A2' / 'ISO 14025'",
    #     'Laenderkennung,DE',
    #     'Typ,specific dataset',
    #     'Referenzjahr,2023',
    #     'Gueltig bis,2028',
    #     'URL,https://www.oekobaudat.de/OEKOBAU.DAT/resource/processes/8ef7b1b9-da21-43c9-a7d1-9a14d9dc7bcf?version=00.01.002',
    #     'Declaration owner,Holcim (Deutschland) GmbH',
    #     'Veroeffentlicht am,2023-10-19',
    #     'Registrierungsnummer,IBU-CEI-HOL-2205120-DE2023000648-1SUG001-DE',
    #     'Registrierungsstelle,Institut Bauen und Umwelt e.V.',
    #     'UUID des Vorgängers,8ef7b1b9-da21-43c9-a7d1-9a14d9dc7bcf',
    #     'Version des Vorgängers,00.01.001',
    #     'URL des Vorgängers,https://www.oekobaudat.de/OEKOBAU.DAT/resource/processes/8ef7b1b9-da21-43c9-a7d1-9a14d9dc7bcf?version=00.01.001',
    # ],
# }

# One Attribute at a time

In [159]:
import json
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Define JSON schema
json_schema = {
    "title": "AlignmentResponse",
    "description": "Response containing alignment mappings.",
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "attribute": {
                "type": "string",
                "description": "The exact attribute name from dataset A without additional information",
            },
            "match_type": {
                "type": "string",
                "description": "The type of SKOS match",
                "enum": ["skos:exactMatch", "skos:closeMatch", "skos:relatedMatch"],
            },
            "field_name": {
                "type": "string",
                "description": "The exact 'Field Name (en)' from Schema B without additional information",
            },
        },
        "required": ["attribute", "match_type", "field_name"],
    },
}


def query_system(attribute, vectorstore_path, schema_filter):
    embeddings = OllamaEmbeddings(model="bge-m3:latest")
    vectorstore = FAISS.load_local(
        vectorstore_path, embeddings=embeddings, allow_dangerous_deserialization=True
    )

    retriever = vectorstore.as_retriever(
        search_kwargs={"filter": {"schema_type": schema_filter}}
    )
    retrieved_docs = retriever.invoke(attribute)

    if not retrieved_docs:
        return None

    context = "\n".join(doc.page_content for doc in retrieved_docs)

    prompt_template = ChatPromptTemplate.from_template(
        """
You are an expert in semantic data alignment and ontology matching. Your task is to map the provided attribute from dataset A to its corresponding attribute in Schema B. Use the SKOS relationship types to indicate the alignment:
- skos:exactMatch: Attributes are identical in meaning.
- skos:closeMatch: Attributes are strongly similar, differing only in minor details.
- skos:relatedMatch: Attributes are conceptually related but not hierarchically or equivalently aligned.

Attribute Schema B:
<headers>
'Field Name (de)','Field Name (en)','Element/Attribute Name','Datatype','Definition (de)','Definition (en)','Original ILCD Format Definition'
</headers>
<context>
{context}
</context>

Answer the following question:
Match the attribute '{attribute}' from dataset A to one and only one attribute from Schema B.

Return the response in JSON format adhering to the defined schema.
"""
    )

    final_prompt = prompt_template.format_prompt(
        context=context, attribute=attribute
    ).to_string()

    # Write the final prompt to a text file
    with open(f"../data/prompts/prompts_{llm_model_name}_1by1.txt", "a") as prompt_file:
        prompt_file.write(f"Final Prompt for {attribute}:\n")
        prompt_file.write(final_prompt + "\n")
        prompt_file.write("-" * 50 + "\n")

    print(final_prompt)

    model = ChatOllama(model=llm_model)
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)

    structured_response = raw_response.get("parsed", None)

    print(structured_response)
    print("-" * 50 + "\n")

    return structured_response


if __name__ == "__main__":
    llm_model = "granite3-dense:8b" # llama3.1:8b, granite3-dense:8b, marco-o1:latest, dolphin3:8b
    llm_model_name = llm_model.split(":")[0]
    vectorstore_path = "../embeddings/bge-m3/row_cs3000_co0_faiss_index"
    output_file = f"../data/responses/response_{llm_model_name}_1by1.json"

    # Reset the JSON file
    with open(output_file, "w") as file:
        json.dump([], file)

    all_responses = []

    for schema, attributes in attribute_schema_mapping.items():
        for attribute in attributes:
            response = query_system(attribute, vectorstore_path, schema)
            if response:
                all_responses.extend(response)

    with open(output_file, "w") as file:
        json.dump(all_responses, file, indent=2)

    print(f"All responses saved to {output_file}")

Human: 
You are an expert in semantic data alignment and ontology matching. Your task is to map the provided attribute from dataset A to its corresponding attribute in Schema B. Use the SKOS relationship types to indicate the alignment:
- skos:exactMatch: Attributes are identical in meaning.
- skos:closeMatch: Attributes are strongly similar, differing only in minor details.
- skos:relatedMatch: Attributes are conceptually related but not hierarchically or equivalently aligned.

Attribute Schema B:
<headers>
'Field Name (de)','Field Name (en)','Element/Attribute Name','Datatype','Definition (de)','Definition (en)','Original ILCD Format Definition'
</headers>
<context>
'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in "Datensatzversion" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the "Data set version", the UUID uniquely 

# All Attributes per Schema File at a time

- Maximal Marginal Relevance (MMR) Retrieval works better thena the default similarity retrieval

In [133]:
import json
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Define JSON schema
json_schema = {
    "title": "AlignmentResponse",
    "description": "Response containing alignment mappings.",
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "attribute": {
                "type": "string",
                "description": "The attribute from dataset A without additional information",
            },
            "match_type": {
                "type": "string",
                "description": "The type of SKOS match",
                "enum": ["skos:exactMatch", "skos:closeMatch", "skos:relatedMatch"],
            },
            "field_name": {
                "type": "string",
                "description": "The exact Field Name (en) from Schema B without additional information",
            },
        },
        "required": ["attribute", "match_type", "field_name"],
    },
}


def query_system(attributes, vectorstore_path, schema_filter):
    embeddings = OllamaEmbeddings(model=embed_model)
    vectorstore = FAISS.load_local(
        vectorstore_path, embeddings=embeddings, allow_dangerous_deserialization=True
    )

    # retriever = vectorstore.as_retriever(
    #     search_kwargs={"filter": {"schema_type": schema_filter}, "k": len(attributes)}
    # )
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "filter": {"schema_type": schema_filter},
            "k": 2 * len(attributes),
            "fetch_k": 50,
            "lambda_mult": 1
        }
    )
    retrieved_docs = retriever.invoke("\n".join(attributes))

    print(f"Number of attributes: {len(attributes)}")
    print(f"Number of retrieved documents: {len(retrieved_docs)}")

    if not retrieved_docs:
        return None

    context = "\n\n".join(doc.page_content for doc in retrieved_docs)

    prompt_template = ChatPromptTemplate.from_template(
        """
You are an expert in semantic data alignment and ontology matching. Your task is to map the provided attributes from dataset A to their corresponding fields in Schema B. Use the SKOS relationship types to indicate the alignment:
- skos:exactMatch: Attributes are identical in meaning.
- skos:closeMatch: Attributes are strongly similar, differing only in minor details.
- skos:relatedMatch: Attributes are conceptually related but not hierarchically or equivalently aligned.

Definition Schema B:
<headers>
'Field Name (de)','Field Name (en)','Element/Attribute Name','Datatype','Definition (de)','Definition (en)','Original ILCD Format Definition'
</headers>
<context>
{context}
</context>

Match the following attributes to the data under <context> in Schema B:
[
{attributes}
]

Return the response in JSON format adhering to the defined schema.
"""
    )

    final_prompt = prompt_template.format_prompt(
        context=context, attributes="\n".join(attributes)
    ).to_string()

    with open(f"{ollama_prompts}", "a") as prompt_file:
        prompt_file.write(final_prompt + "\n\n" + ("-" * 50) + "\n\n")

    print(final_prompt)

    model = ChatOllama(model=llm_model)
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)

    structured_response = raw_response.get("parsed", None)

    print(structured_response)
    print("-" * 50 + "\n")

    return structured_response


if __name__ == "__main__":
    llm_model = "dolphin3:8b" # llama3.1:8b, granite3-dense:8b, marco-o1:latest
    embed_model = "snowflake-arctic-embed2:latest" # bge-m3:latest

    if embed_model == "bge-m3:latest" or embed_model == "bge-large:latest":
        embed_model_name = embed_model.split(":")[0].replace("/", "_")
    else:
        embed_model_name = embed_model.split("-")[0].replace("/", "_")  
    chunk_strat = "row_cs1000_co0"
    vectorstore_path = f"../embeddings/{embed_model_name}/{chunk_strat}_faiss_index"
    output_file = f"../data/responses/response_{llm_model.split(":")[0]}.json"

    # Reset the JSON file
    with open(output_file, "w", encoding='utf-8') as file:
        json.dump([], file)

    # Reset the prompts text file
    ollama_prompts = "../data/prompts/prompts_ollama_csv02.txt"
    with open(f"{ollama_prompts}", "w") as prompt_file:
        prompt_file.write("")

    all_responses = []

    for schema, attributes in attribute_schema_mapping.items():
        response = query_system(attributes, vectorstore_path, schema)
        if response:
            all_responses.extend(response)

    with open(output_file, "w", encoding='utf-8') as file:
        json.dump(all_responses, file, indent=2)

    print(f"All responses saved to {output_file}")


Number of attributes: 8
Number of retrieved documents: 16
Human: 
You are an expert in semantic data alignment and ontology matching. Your task is to map the provided attributes from dataset A to their corresponding fields in Schema B. Use the SKOS relationship types to indicate the alignment:
- skos:exactMatch: Attributes are identical in meaning.
- skos:closeMatch: Attributes are strongly similar, differing only in minor details.
- skos:relatedMatch: Attributes are conceptually related but not hierarchically or equivalently aligned.

Definition Schema B:
<headers>
'Field Name (de)','Field Name (en)','Element/Attribute Name','Datatype','Definition (de)','Definition (en)','Original ILCD Format Definition'
</headers>
<context>
'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in "Datensatzversion" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data se

### Similarity Score

In [42]:
import json
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Define JSON schema
json_schema = {
    "title": "AlignmentResponse",
    "description": "Response containing alignment mappings.",
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "attribute": {
                "type": "string",
                "description": "The attribute from dataset A without additional information",
            },
            "match_type": {
                "type": "string",
                "description": "The type of SKOS match",
                "enum": ["skos:exactMatch", "skos:closeMatch", "skos:relatedMatch"],
            },
            "field_name": {
                "type": "string",
                "description": "The exact Field Name (en) from Schema B without additional information",
            },
        },
        "required": ["attribute", "match_type", "field_name"],
    },
}


def query_system(attributes, vectorstore_path, schema_filter):
    embeddings = OllamaEmbeddings(model="bge-m3:latest")
    vectorstore = FAISS.load_local(
        vectorstore_path, embeddings=embeddings, allow_dangerous_deserialization=True
    )

    # Retrieve documents with scores
    retrieved_docs_with_scores = vectorstore.similarity_search_with_score(
        "\n".join(attributes), k=2 * len(attributes), search_kwargs={
            "filter": {"schema_type": schema_filter},}
    )

    print(f"Number of attributes: {len(attributes)}")
    print(f"Number of retrieved documents: {len(retrieved_docs_with_scores)}")

    # Print retrieved document scores
    for doc, score in retrieved_docs_with_scores:
        print(f"Score: {score:.4f}")
        print(f"Content: {doc.page_content}")
        print("-" * 50)

    if not retrieved_docs_with_scores:
        return None

    context = "\n\n".join(doc.page_content for doc, _ in retrieved_docs_with_scores)

    prompt_template = ChatPromptTemplate.from_template(
        """
You are an expert in semantic data alignment and ontology matching. Your task is to map the provided attributes from dataset A to their corresponding fields in Schema B. Use the SKOS relationship types to indicate the alignment:
- skos:exactMatch: Attributes are identical in meaning.
- skos:closeMatch: Attributes are strongly similar, differing only in minor details.
- skos:relatedMatch: Attributes are conceptually related but not hierarchically or equivalently aligned.

Definition Schema B:
<headers>
'Field Name (de)','Field Name (en)','Element/Attribute Name','Datatype','Definition (de)','Definition (en)','Original ILCD Format Definition'
</headers>
<context>
{context}
</context>

Match the following attributes to the data under <context> in Schema B:
[Attribute,Value
{attributes}
]

Return the response in JSON format adhering to the defined schema.
"""
    )

    final_prompt = prompt_template.format_prompt(
        context=context, attributes="\n".join(attributes)
    ).to_string()

    with open(f"{ollama_prompts}", "a", encoding='utf-8') as prompt_file:
        prompt_file.write(final_prompt + "\n\n" + ("-" * 50) + "\n\n")

    print(final_prompt)

    model = ChatOllama(model="dolphin3:8b")

    # Add structured output to the LLM model
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)

    structured_response = raw_response.get("parsed", None)

    print(structured_response)
    print("-" * 50 + "\n")

    return structured_response


if __name__ == "__main__":
    embed_model_name = ""
    vectorstore_path = "../embeddings/bge-m3_row_cs3000_faiss_index"
    output_file = "../data/responses/response_ollama.json"

    # Reset the JSON file
    with open(output_file, "w", encoding='utf-8') as file:
        json.dump([], file)

    # Reset the prompts text file
    ollama_prompts = "../data/prompts/prompts_ollama_csv02.txt"
    with open(f"{ollama_prompts}", "w", encoding='utf-8') as prompt_file:
        prompt_file.write("")

    all_responses = []

    for schema, attributes in attribute_schema_mapping.items():
        response = query_system(attributes, vectorstore_path, schema)
        if response:
            all_responses.extend(response)

    with open(output_file, "w", encoding='utf-8') as file:
        json.dump(all_responses, file, indent=2)

    print(f"All responses saved to {output_file}")


Number of attributes: 19
Number of retrieved documents: 38
Score: 1.0038
Content: 'Hintergrundbericht','Data set LCA report, background info','referenceToExternalDocumentation','GlobalReferenceType','Hier können relevante Dokumente wie z.B. Sicherheitsdatenblätter, Hintergrundbericht o.ä. angehängt werden','Relevant information such as safety declarations or background report can be attached','"Source data set(s)" of detailed LCA study on the process or product represented by this data set, as well as documents / files with overarching documentative information on technology, geographical and / or time aspects etc. (e.g. basic engineering studies, process simulation results, patents, plant documentation, model behind the parameterisation of the "Mathematical model" section, etc.) (Note: can indirectly reference to digital file.)'
--------------------------------------------------
Score: 1.0155
Content: 'Datenquellen','Data source(s) used for this data set','referenceToDataSource','Glob

# Embedding Models Comparison 

In [73]:
# Define all expected chunks
expected_chunks = [
    "'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in \"Datensatzversion\" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the \"Data set version\", the UUID uniquely identifies each data set.'",
    "'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer des Datensatzes','~','Version number of data set. First two digits refer to major updates, the second two digits to minor revisions and error corrections etc. The third three digits are intended for automatic and internal counting of versions during data set development. Together with the data set's UUID, the \"Data set version\" uniquely identifies each data set.'",
    "'Name','Name','baseName','StringMultiLang','Allgemeiner Name des Produkts oder Systems','~','General descriptive name of the process and/or its main good(s) or service(s) and/or its level of processing.'",
    "'Klassifizierung','Classification','classification','nan','ein Gliederungssystem mit Gliederungsklassen','~','Optional statistical or other classification of the data set. Typically also used for structuring LCA databases.'",
    "'Konformität','Compliance','compliance','nan','eine Konformitätsdeklaration','~','one compliance declaration'",
    "'Ort','Location','@location','NullableString','Region, für die der Datensatz repräsentativ ist ISO 3166-Ländercode oder Regionalcode','Region, for which the data set is representative / relevant. ISO 3166 country code or regional code','Location, country or region the data set represents. [Note 1: This field does not refer to e.g. the country in which a specific site is located that is represented by this data set but to the actually represented country, region, or site. Note 2: Entry can be of type \"two-letter ISO 3166 country code\" for countries, \"seven-letter regional codes\" for regions or continents, or \"market areas and market organisations\", as predefined for the ILCD. Also a name for e.g. a specific plant etc. can be given here (e.g. \"FR, Lyon, XY Company, Z Site\"; user defined). Note 3: The fact whether the entry refers to production or to consumption / supply has to be stated in the name-field \"Mix and location types\" e.g. as \"Production mix\".]'",
    "'Subtyp','Subtype','epd:subType','generic datasetGenerischer Datensatzrepresentative datasetRepräsentativer Datensatzaverage datasetDurchschnittsdatensatzspecific dataset(Hersteller-) Spezifischer Datensatztemplate datasetMusterdatensatz','Gibt den Datensatztypen hinsichtlich Repräsentativität an. Einer der folgenden vordefinierten Datensatztypen muss ausgewählt werden: -specific dataset(spezifischer Datensatz) - hersteller-(unternehmens-) spezifischer Datensatz für ein konkretes Produkt eines Werkes -average dataset(Durchschnittsdatensatz) - durchschnittliche Datensätze von Industrieverbänden, mehreren Firmen, mehreren Werken oder mehreren Produkten (d.h. auf Grundlage von Daten der Industrieproduktion von Unternehmen) -representative dataset(repräsentativer Datensatz) - representative dataset – Daten, die repräsentativ für ein Land / eine Region sind (z.B. Durchschnitt DE) -template dataset(Muster-EPD-Datensatz) - unspezifische Datensätze für spezifische Produkte, die auf Basis einer „Muster-EPD“ erstellt wurden -generic dataset(generischer Datensatz) - generische Daten gemäß EN 15804 sowie andere, nicht auf Basis von Industriedaten modellierte Daten (z.B. auf der Basis von Literatur, Expertenwissen etc.)','Indicates the type of data set regarding its representativeness. One of the following predefined data types has to be chosen: -specific dataset- vendor (company) specific data for a specific product from one production site -average dataset- avarage datasets from industry associations, multiple manufacturers, multiple production sites or multiple products, i.e. modelled based on industry data from an manufacturer -representative dataset- data that is representative for a country or region (e.g. average for Germany) -template dataset- sample EPD, unspecific datasets for specific products, that were created based on a sample EPD -generic dataset- generic data acc. to EN 15804 and data based on other non-industry data sources (e.g. literature, expert knowledge)','nan'",
    "'Referenzjahr','Reference year','referenceYear','Year','Das erste Jahr der Gültigkeits des Datensatzes.','Start year of the time period for which the data set is valid (until year of \"Data set valid until\")','Start year of the time period for which the data set is valid (until year of \"Data set valid until:\"). For data sets that combine data from different years, the most representative year is given regarding the overall environmental impact. In that case, the reference year is derived by expert judgement.'",
    "'Gültig bis','Data set valid until:','dataSetValidUntil','Year','Ende des Zeitabschnitts, bis zu dem der Datensatz gültig ist.','End year of the time period for which the data set is valid.','End year of the time period for which the data set is still valid / sufficiently representative. This date also determines when a data set revision / remodelling is required or recommended due to expected relevant changes in environmentally or technically relevant inventory values, including in the background system.'",
    "'Permanente Datensatz-URI','Permanent data set URI','permanentDataSetURI','anyURI','URI zum Original dieses Datensatzes','~','URI (i.e. an internet address) of the original of this data set. [Note: This equally globally unique identifier supports users and software tools to identify and retrieve the original version of a data set via the internet or to check for available updates. The URI must not represent an existing WWW address, but it should be unique and point to the data access point, e.g. by combining the data owner's www path with the data set's UUID, e.g. http://www.mycompany.com/lca/processes/50f12420-8855-12db-b606-0900210c9a66.]'",
    "'Eigner des Datensatzes','Owner of data set','referenceToOwnershipOfDataSet','GlobalReferenceType','\"Contact\"-Datensatz zum Eigentümer des Datensatzes','~','\"Contact data set\" of the person or entity who owns this data set. (Note: this is not necessarily the publisher of the data set.)'",
    "'Veröffentlichungsdatum der EPD','publication date of EPD','publicationDateOfEPD','xs:date','Datum der Veröffentlichung der EPD in der Form \"YYYY-MM-DD\".','Exact date of publication of the EPD in the form \"YYYY-MM-DD\".','nan'",
    "'Registrierungsnummer','Registration number','registrationNumber','String','ID-Nummer der EPD oder des Projekts','ID number of EPD or project','A unique identifying number for this data set issued by the registration authority.'",
    "'Herausgeber','Issuer','referenceToRegistrationAuthority','GlobalReferenceType','Kontaktdaten des Herausgebers des Datensatzes (z.B. EPD-Programmbetreiber)','~','\"Contact data set\" of the authority that has registered this data set.'",
    "'Vorhergehende Datensatzversion','Preceding data set version','referenceToPrecedingDataSetVersion','GlobalReferenceType','nan','nan','Last preceding data set, which was replaced by this version. Either a URI of that data set (i.e. an internet address) or its UUID plus version number is given (or both).'",
    "'Modul/Phase','Module/Phase','@epd:module','String','Modul oder Phase (z.B. \"A1-A3\")','Module or phase (e.g. \"A1-A3\")','nan'",
    "'Szenario','Scenario','@epd:scenario','String','Verweis auf die oben definierte ID eines Szenarios (falls definiert), für das dieser Wert gilt.','References ID of a scenario defined above','nan'",
    "'Beschreibung','Description','epd:description','FTMultiLang','Beschreibung des Szenarios','Description of the scenario','nan'"
]


In [3]:
# Define 4 expected chunks
expected_chunks = [
    "'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in \"Datensatzversion\" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the \"Data set version\", the UUID uniquely identifies each data set.'",
    "'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer des Datensatzes','~','Version number of data set. First two digits refer to major updates, the second two digits to minor revisions and error corrections etc. The third three digits are intended for automatic and internal counting of versions during data set development. Together with the data set's UUID, the \"Data set version\" uniquely identifies each data set.'",
    "'Name','Name','baseName','StringMultiLang','Allgemeiner Name des Produkts oder Systems','~','General descriptive name of the process and/or its main good(s) or service(s) and/or it's level of processing.'",
    "'Klassifizierung','Classification','classification','nan','ein Gliederungssystem mit Gliederungsklassen','~','Optional statistical or other classification of the data set. Typically also used for structuring LCA databases.'",
    "'Konformität','Compliance','compliance','nan','eine Konformitätsdeklaration','~','one compliance declaration'",
    "'Ort','Location','@location','NullableString','Region, für die der Datensatz repräsentativ ist ISO 3166-Ländercode oder Regionalcode','Region, for which the data set is representative / relevant. ISO 3166 country code or regional code','Location, country or region the data set represents. [Note 1: This field does not refer to e.g. the country in which a specific site is located that is represented by this data set but to the actually represented country, region, or site. Note 2: Entry can be of type \"two-letter ISO 3166 country code\" for countries, \"seven-letter regional codes\" for regions or continents, or \"market areas and market organisations\", as predefined for the ILCD. Also a name for e.g. a specific plant etc. can be given here (e.g. \"FR, Lyon, XY Company, Z Site\"; user defined). Note 3: The fact whether the entry refers to production or to consumption / supply has to be stated in the name-field \"Mix and location types\" e.g. as \"Production mix\".]'",
]

In [172]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# Replace with your actual attribute->schema mapping
attribute_schema_mapping = {
    "EPD_DataSet": [
        "UUID",
        "Version",
        "Name (de)",
        "Name (en)",
        "Kategorie (original)",
        "Kategorie (en)",
        "Konformität",
        "Laenderkennung",
    ]
}

# Replace with your actual expected chunks
expected_chunks = [
    "'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in \"Datensatzversion\" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the \"Data set version\", the UUID uniquely identifies each data set.'",
    "'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer des Datensatzes','~','Version number of data set. First two digits refer to major updates, the second two digits to minor revisions and error corrections etc. The third three digits are intended for automatic and internal counting of versions during data set development. Together with the data set's UUID, the \"Data set version\" uniquely identifies each data set.'",
    "'Name','Name','baseName','StringMultiLang','Allgemeiner Name des Produkts oder Systems','~','General descriptive name of the process and/or its main good(s) or service(s) and/or it's level of processing.'",
    "'Klassifizierung','Classification','classification','nan','ein Gliederungssystem mit Gliederungsklassen','~','Optional statistical or other classification of the data set. Typically also used for structuring LCA databases.'",
    "'Konformität','Compliance','compliance','nan','eine Konformitätsdeklaration','~','one compliance declaration'",
    "'Ort','Location','@location','NullableString','Region, für die der Datensatz repräsentativ ist ISO 3166-Ländercode oder Regionalcode','Region, for which the data set is representative / relevant. ISO 3166 country code or regional code','Location, country or region the data set represents. [Note 1: This field does not refer to e.g. the country in which a specific site is located that is represented by this data set but to the actually represented country, region, or site. Note 2: Entry can be of type \"two-letter ISO 3166 country code\" for countries, \"seven-letter regional codes\" for regions or continents, or \"market areas and market organisations\", as predefined for the ILCD. Also a name for e.g. a specific plant etc. can be given here (e.g. \"FR, Lyon, XY Company, Z Site\"; user defined). Note 3: The fact whether the entry refers to production or to consumption / supply has to be stated in the name-field \"Mix and location types\" e.g. as \"Production mix\".]'",
]


def compare_models(attributes, vectorstore_paths, schema):
    """
    Compare multiple models for a given schema by retrieving documents
    and evaluating them with exact-match and rank-based metrics. 
    Then, print a final Markdown summary table.
    """
    query = "\n".join(attributes)

    # We'll store each model's evaluation results here
    evaluation_data = []

    for model_name, path in vectorstore_paths.items():
        print(f"\nRetrieving documents using {model_name}:")
        if model_name == "jinaai/jina-embeddings-v3" or model_name == "HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1.5":
            embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"trust_remote_code": True}
    )
        else:
            embeddings = OllamaEmbeddings(model=model_name)

        vectorstore = FAISS.load_local(
            path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True,  
        )

        # Example MMR-based retriever
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "filter": {"schema_type": schema},
                "k": k_value,  # Adjust 'k' as needed
                "fetch_k": 100,
                "lambda_mult": 1,
            },
        )

        # Retrieve documents
        retrieved_docs = retriever.invoke(query)
        print(f"Number of retrieved documents: {len(retrieved_docs)}")

        # ---- Exact-Match Evaluation ----
        retrieved_chunks = [doc.page_content for doc in retrieved_docs]
        correctly_retrieved, missed_chunks = evaluate_retrieval(retrieved_chunks)

        # ---- Ranked Evaluation ----
        # Turn the retrieved docs into (rank, Document) for checking rank
        docs_with_ranks = list(enumerate(retrieved_docs, start=1))
        mean_rank, mrr = evaluate_ranked_retrieval(docs_with_ranks)

        # ---- Store Results ----
        evaluation_data.append({
            "model": model_name,
            "retrieved_docs": len(retrieved_docs),
            "found_exact": len(correctly_retrieved),
            "missed_exact": len(missed_chunks),
            "mean_rank": mean_rank,
            "mrr": mrr,
        })

    # After processing all models, print the Markdown table
    print_markdown_table(evaluation_data)


def evaluate_retrieval(retrieved_chunks):
    """
    Exact match check: do we see the entire chunk text in the retrieved list?
    Also prints a quick overview.
    """
    correctly_retrieved = [c for c in expected_chunks if c in retrieved_chunks]
    missed_chunks = [c for c in expected_chunks if c not in retrieved_chunks]

    print(f"\nExact-Match Evaluation:")
    print(f"  Correctly retrieved chunks: {len(correctly_retrieved)}/{len(expected_chunks)}")
    for chunk in correctly_retrieved:
        print(f"    ✔ Retrieved: {chunk}")

    print(f"  Missed chunks: {len(missed_chunks)}/{len(expected_chunks)}")
    for chunk in missed_chunks:
        print(f"    ✘ Missed: {chunk}")

    return correctly_retrieved, missed_chunks


def evaluate_ranked_retrieval(docs_with_ranks):
    """
    Rank-based check: for each expected chunk, see at which rank it appears
    in the retrieved list, then compute Mean Rank and MRR.
    Also prints a quick overview.
    """
    ranks = []
    for chunk in expected_chunks:
        matched_positions = [
            rank
            for (rank, doc) in docs_with_ranks
            if doc.page_content.strip() == chunk.strip()
        ]
        # If found, take first match
        if matched_positions:
            ranks.append(matched_positions[0])
        else:
            ranks.append(None)

    found_ranks = [r for r in ranks if r is not None]
    missed_count = sum(r is None for r in ranks)

    if found_ranks:
        mean_rank = sum(found_ranks) / len(found_ranks)
        mrr = sum((1.0 / r) for r in found_ranks) / len(found_ranks)
    else:
        mean_rank = None
        mrr = None

    print(f"\nRanked Evaluation:")
    print(f"  Found: {len(found_ranks)}/{len(expected_chunks)}")
    print(f"  Missed: {missed_count}/{len(expected_chunks)}")
    print(f"  Mean Rank: {mean_rank}")
    print(f"  MRR: {mrr}")

    return mean_rank, mrr


def print_markdown_table(evaluation_data):
    """
    Print all evaluation data in a final Markdown table for easy comparison.
    """
    print(f"\n## Final Results (Markdown Table) k={k_value}\n")
    print("| Model | Retrieved Docs | Found Exact | Missed Exact | Mean Rank | MRR |")
    print("|---|---|---|---|---|---|")
    for row in evaluation_data:
        # Handle None values gracefully
        mean_rank_str = f"{row['mean_rank']:.2f}" if row["mean_rank"] else "N/A"
        mrr_str = f"{row['mrr']:.3f}" if row["mrr"] else "N/A"

        print(
            f"| {row['model']} "
            f"| {row['retrieved_docs']} "
            f"| {row['found_exact']} "
            f"| {row['missed_exact']} "
            f"| {mean_rank_str} "
            f"| {mrr_str} |"
        )


if __name__ == "__main__":
    k_value = 5 * len(attributes)
    embeddings_name = "row_cs3000_co0_faiss_index"
    vectorstore_paths = {
        "bge-m3:latest": f"../embeddings/bge-m3/{embeddings_name}",
        "snowflake-arctic-embed2:latest": f"../embeddings/snowflake/{embeddings_name}",
        "jina/jina-embeddings-v2-base-de:latest": f"../embeddings/jina_jina/{embeddings_name}",
        "paraphrase-multilingual:latest": f"../embeddings/paraphrase/{embeddings_name}",
        "jinaai/jina-embeddings-v3": f"../embeddings/jinaai_jina-embeddings-v3/{embeddings_name}_COSINE",
        "HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1.5": f"../embeddings/HIT-TMG_KaLM-embedding-multilingual-mini-instruct-v1.5/{embeddings_name}",
        "jeffh/intfloat-multilingual-e5-large-instruct:f32": "../embeddings/jeffh_intfloat/row_cs1000_co0_faiss_index",
        "granite-embedding:278m": "../embeddings/granite/row_cs1000_co0_faiss_index",
        "bge-large:latest": "../embeddings/bge-large/row_cs1000_co0_faiss_index",
        
    }

    for schema, attributes in attribute_schema_mapping.items():
        compare_models(attributes, vectorstore_paths, schema)



Retrieving documents using bge-m3:latest:
Number of retrieved documents: 40

Exact-Match Evaluation:
  Correctly retrieved chunks: 5/6
    ✔ Retrieved: 'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in "Datensatzversion" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the "Data set version", the UUID uniquely identifies each data set.'
    ✔ Retrieved: 'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer des Datensatzes','~','Version number of data set. First two digits refer to major updates, the second two digits to minor revisions and error corrections etc. The third three digits are intended for automatic and internal counting of versions during data set development. Together with the data set's UUID, the "Data set version" uniquely identifies each data set.'
    ✔ Retrieved: 'Name','Name','b

In [173]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# Replace with your actual attribute->schema mapping
attribute_schema_mapping = {
    "EPD_DataSet": [
        "UUID",
        "Version",
        "Name (de)",
        "Name (en)",
        "Kategorie (original)",
        "Kategorie (en)",
        "Konformität",
        "Laenderkennung",
    ]
}

# Replace with your actual expected chunks
expected_chunks = [
    "'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in \"Datensatzversion\" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the \"Data set version\", the UUID uniquely identifies each data set.'",
    "'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer des Datensatzes','~','Version number of data set. First two digits refer to major updates, the second two digits to minor revisions and error corrections etc. The third three digits are intended for automatic and internal counting of versions during data set development. Together with the data set's UUID, the \"Data set version\" uniquely identifies each data set.'",
    "'Name','Name','baseName','StringMultiLang','Allgemeiner Name des Produkts oder Systems','~','General descriptive name of the process and/or its main good(s) or service(s) and/or it's level of processing.'",
    "'Klassifizierung','Classification','classification','nan','ein Gliederungssystem mit Gliederungsklassen','~','Optional statistical or other classification of the data set. Typically also used for structuring LCA databases.'",
    "'Konformität','Compliance','compliance','nan','eine Konformitätsdeklaration','~','one compliance declaration'",
    "'Ort','Location','@location','NullableString','Region, für die der Datensatz repräsentativ ist ISO 3166-Ländercode oder Regionalcode','Region, for which the data set is representative / relevant. ISO 3166 country code or regional code','Location, country or region the data set represents. [Note 1: This field does not refer to e.g. the country in which a specific site is located that is represented by this data set but to the actually represented country, region, or site. Note 2: Entry can be of type \"two-letter ISO 3166 country code\" for countries, \"seven-letter regional codes\" for regions or continents, or \"market areas and market organisations\", as predefined for the ILCD. Also a name for e.g. a specific plant etc. can be given here (e.g. \"FR, Lyon, XY Company, Z Site\"; user defined). Note 3: The fact whether the entry refers to production or to consumption / supply has to be stated in the name-field \"Mix and location types\" e.g. as \"Production mix\".]'",
]


def compare_models(attributes, vectorstore_paths, schema):
    """
    Compare multiple models for a given schema by retrieving documents
    and evaluating them with exact-match and rank-based metrics. 
    Then, print a final Markdown summary table.
    """
    query = "\n".join(attributes)

    # We'll store each model's evaluation results here
    evaluation_data = []

    for model_name, path in vectorstore_paths.items():
        print(f"\nRetrieving documents using {model_name}:")
        if model_name == "jinaai/jina-embeddings-v3" or model_name == "HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1.5":
            embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"trust_remote_code": True}
    )
        else:
            embeddings = OllamaEmbeddings(model=model_name)

        vectorstore = FAISS.load_local(
            path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True,  
        )

        # Example MMR-based retriever
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "filter": {"schema_type": schema},
                "k": k_value,  # Adjust 'k' as needed
                "fetch_k": 100,
                "lambda_mult": 1,
            },
        )

        # Retrieve documents
        retrieved_docs = retriever.invoke(query)
        print(f"Number of retrieved documents: {len(retrieved_docs)}")

        # ---- Exact-Match Evaluation ----
        retrieved_chunks = [doc.page_content for doc in retrieved_docs]
        correctly_retrieved, missed_chunks = evaluate_retrieval(retrieved_chunks)

        # ---- Ranked Evaluation ----
        # Turn the retrieved docs into (rank, Document) for checking rank
        docs_with_ranks = list(enumerate(retrieved_docs, start=1))
        mean_rank, mrr = evaluate_ranked_retrieval(docs_with_ranks)

        # ---- Store Results ----
        evaluation_data.append({
            "model": model_name,
            "retrieved_docs": len(retrieved_docs),
            "found_exact": len(correctly_retrieved),
            "missed_exact": len(missed_chunks),
            "mean_rank": mean_rank,
            "mrr": mrr,
        })

    # After processing all models, print the Markdown table
    print_markdown_table(evaluation_data)


def evaluate_retrieval(retrieved_chunks):
    """
    Exact match check: do we see the entire chunk text in the retrieved list?
    Also prints a quick overview.
    """
    correctly_retrieved = [c for c in expected_chunks if c in retrieved_chunks]
    missed_chunks = [c for c in expected_chunks if c not in retrieved_chunks]

    print(f"\nExact-Match Evaluation:")
    print(f"  Correctly retrieved chunks: {len(correctly_retrieved)}/{len(expected_chunks)}")
    for chunk in correctly_retrieved:
        print(f"    ✔ Retrieved: {chunk}")

    print(f"  Missed chunks: {len(missed_chunks)}/{len(expected_chunks)}")
    for chunk in missed_chunks:
        print(f"    ✘ Missed: {chunk}")

    return correctly_retrieved, missed_chunks


def evaluate_ranked_retrieval(docs_with_ranks):
    """
    Rank-based check: for each expected chunk, see at which rank it appears
    in the retrieved list, then compute Mean Rank and MRR.
    Also prints a quick overview.
    """
    ranks = []
    for chunk in expected_chunks:
        matched_positions = [
            rank
            for (rank, doc) in docs_with_ranks
            if doc.page_content.strip() == chunk.strip()
        ]
        # If found, take first match
        if matched_positions:
            ranks.append(matched_positions[0])
        else:
            ranks.append(None)

    found_ranks = [r for r in ranks if r is not None]
    missed_count = sum(r is None for r in ranks)

    if found_ranks:
        mean_rank = sum(found_ranks) / len(found_ranks)
        mrr = sum((1.0 / r) for r in found_ranks) / len(found_ranks)
    else:
        mean_rank = None
        mrr = None

    print(f"\nRanked Evaluation:")
    print(f"  Found: {len(found_ranks)}/{len(expected_chunks)}")
    print(f"  Missed: {missed_count}/{len(expected_chunks)}")
    print(f"  Mean Rank: {mean_rank}")
    print(f"  MRR: {mrr}")

    return mean_rank, mrr


def print_markdown_table(evaluation_data):
    """
    Print all evaluation data in a final Markdown table for easy comparison.
    """
    print(f"\n## Final Results (Markdown Table) k={k_value}\n")
    print("| Model | Retrieved Docs | Found Exact | Missed Exact | Mean Rank | MRR |")
    print("|---|---|---|---|---|---|")
    for row in evaluation_data:
        # Handle None values gracefully
        mean_rank_str = f"{row['mean_rank']:.2f}" if row["mean_rank"] else "N/A"
        mrr_str = f"{row['mrr']:.3f}" if row["mrr"] else "N/A"

        print(
            f"| {row['model']} "
            f"| {row['retrieved_docs']} "
            f"| {row['found_exact']} "
            f"| {row['missed_exact']} "
            f"| {mean_rank_str} "
            f"| {mrr_str} |"
        )


if __name__ == "__main__":
    k_values = [3 * len(attributes), 4 * len(attributes), 5 * len(attributes), 6 * len(attributes), 7 * len(attributes), 8 * len(attributes), 9 * len(attributes), 10 * len(attributes)]
    embeddings_name = "row_cs3000_co0_faiss_index"
    vectorstore_paths = {
        "bge-m3:latest": f"../embeddings/bge-m3/{embeddings_name}",
        "snowflake-arctic-embed2:latest": f"../embeddings/snowflake/{embeddings_name}",
        "jina/jina-embeddings-v2-base-de:latest": f"../embeddings/jina_jina/{embeddings_name}",
        "paraphrase-multilingual:latest": f"../embeddings/paraphrase/{embeddings_name}",
        "jinaai/jina-embeddings-v3": f"../embeddings/jinaai_jina-embeddings-v3/{embeddings_name}_COSINE",
        "HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1.5": f"../embeddings/HIT-TMG_KaLM-embedding-multilingual-mini-instruct-v1.5/{embeddings_name}",
        "jeffh/intfloat-multilingual-e5-large-instruct:f32": "../embeddings/jeffh_intfloat/row_cs1000_co0_faiss_index",
        "granite-embedding:278m": "../embeddings/granite/row_cs1000_co0_faiss_index",
        "bge-large:latest": "../embeddings/bge-large/row_cs1000_co0_faiss_index",
        
    }

    for k_value in k_values:
        for schema, attributes in attribute_schema_mapping.items():
            compare_models(attributes, vectorstore_paths, schema)



Retrieving documents using bge-m3:latest:
Number of retrieved documents: 24

Exact-Match Evaluation:
  Correctly retrieved chunks: 3/6
    ✔ Retrieved: 'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datensatzes. Zusammen mit der Versionsnummer in "Datensatzversion" wird der Datensatz damit eindeutig identifizert','~','Automatically generated Universally Unique Identifier of this data set. Together with the "Data set version", the UUID uniquely identifies each data set.'
    ✔ Retrieved: 'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer des Datensatzes','~','Version number of data set. First two digits refer to major updates, the second two digits to minor revisions and error corrections etc. The third three digits are intended for automatic and internal counting of versions during data set development. Together with the data set's UUID, the "Data set version" uniquely identifies each data set.'
    ✔ Retrieved: 'Konformität','C

## Retrieval with rerank
- reranking does not seem to make a difference in the retrieval or prompt

In [123]:
# Rerank retrieved chunks
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from typing import List, Tuple
import numpy as np


def compare_models(attributes, vectorstore_paths, schema):
    """
    Compare multiple models with a two-stage retrieval: broad first pass,
    then re-rank to get the final subset.
    """
    query = "\n".join(attributes)

    for model_name, path in vectorstore_paths.items():
        print(f"\n--- Using model: {model_name} ---")
        embeddings = OllamaEmbeddings(model=model_name)
        vectorstore = FAISS.load_local(
            path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )

        # -------------------------------
        # First Pass: broad retrieval
        # -------------------------------
        # e.g., k=80 to capture as many relevant docs as possible
        docs_with_scores = vectorstore.similarity_search_with_score(
            query, k=172,
            search_kwargs={"filter": {"schema_type": schema}}
        )
        print(f"First-pass retrieved docs: {len(docs_with_scores)}")
        

        # -------------------------------
        # Second Pass: re-rank
        # -------------------------------
        re_ranked = second_pass_rerank(query, docs_with_scores, embeddings)
        
        # Let’s take top 10 after re-ranking
        top_10_reranked = re_ranked[:20]
        print(f"Second-pass top-10 docs after re-ranking:")
        for i, (doc, new_score) in enumerate(top_10_reranked, start=1):
            print(f"  {i}. Score={new_score:.4f} | {doc.page_content[:60]}...")

        # Next, you’d feed just `top_10_reranked` to the LLM for mapping
        # rather than all 80.

        # (Optional) Evaluate against expected chunks
        final_retrieved_chunks = [doc.page_content for doc, _ in top_10_reranked]
        evaluate_retrieval(model_name, final_retrieved_chunks)


def second_pass_rerank(
    query: str,
    docs_with_scores: List[Tuple],
    embedding_function: OllamaEmbeddings
) -> List[Tuple]:
    """
    Re-rank a list of (Document, score) pairs by computing a new similarity
    score. For example, re-embed doc + query and compute dot product or
    cosine similarity. Return a sorted list descending by new score.
    """
    query_emb = embedding_function.embed_query(query)
    new_ranked_results = []

    for doc, _old_score in docs_with_scores:
        # Option A: Re-embed doc text
        doc_emb = embedding_function.embed_query(doc.page_content)
        # Compute dot product (or use cosine similarity)
        score = np.dot(query_emb, doc_emb)
        new_ranked_results.append((doc, score))

    # Sort descending by the new score
    new_ranked_results.sort(key=lambda x: x[1], reverse=True)
    return new_ranked_results


def evaluate_retrieval(model_name, retrieved_chunks):
    """
    Simple exact-match check just to illustrate results.
    """
    print(f"\nEvaluating final retrieval for {model_name}:")
    correctly_retrieved = [c for c in expected_chunks if c in retrieved_chunks]
    missed_chunks = [c for c in expected_chunks if c not in retrieved_chunks]
    print(f"  Found: {len(correctly_retrieved)} / {len(expected_chunks)}")
    print(f"  Missed: {len(missed_chunks)} / {len(expected_chunks)}")


if __name__ == "__main__":
    schema = "EPD_DataSet"
    embeddings_name = "row_cs1000_co0_faiss_index"
    vectorstore_paths = {
        "bge-m3:latest": f"../embeddings/bge-m3/{embeddings_name}",
        "snowflake-arctic-embed2:latest": f"../embeddings/snowflake/{embeddings_name}",
        # ...
    }

    for schema, attributes in attribute_schema_mapping.items():
        compare_models(attributes, vectorstore_paths, schema)



--- Using model: bge-m3:latest ---
First-pass retrieved docs: 172
Second-pass top-10 docs after re-ranking:
  1. Score=0.5941 | 'Name des Konformitätssystems','Compliance system name','ref...
  2. Score=0.5862 | 'Klassenname','Class name','class','nan','Name der Gliederun...
  3. Score=0.5822 | 'Validierung','Validation','validation','nan','Validierungsi...
  4. Score=0.5821 | 'Registrierungsnummer','Registration number','registrationNu...
  5. Score=0.5767 | 'Name','Name','epd2:name','StringMultiLang','Name des Materi...
  6. Score=0.5668 | 'Eindeutiger Klassenidentifizierer','Unique class identifier...
  7. Score=0.5650 | 'Vorhergehende Datensatzversion','Preceding data set version...
  8. Score=0.5650 | 'Klassifizierungsinformation','Classification information','...
  9. Score=0.5630 | 'Konformität','Compliance','compliance','nan','eine Konformi...
  10. Score=0.5628 | 'Name','Name','name','nan','Beschreibender spezifischer Name...
  11. Score=0.5623 | 'Einheitengruppe','Unit group

In [150]:
import numpy as np
from typing import List, Tuple
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_ollama import OllamaEmbeddings

# -------------------------------------------------------------------------------------
# MAIN COMPARISON FUNCTION
# -------------------------------------------------------------------------------------
def two_stage_retrieval_and_eval(attributes, vectorstore_paths, schema, k_value=40):
    """
    1) MMR-based retrieval (k = k_value)
    2) Print the MMR retrieval
    3) Re-rank those docs with a second pass
    4) Print the re-ranked retrieval
    5) Evaluate coverage of final top docs
    """
    query = "\n".join(attributes)
    top_n_to_print = 20  # How many docs to show in each list for demonstration
    top_n_for_eval = 20  # How many docs to consider "final" for coverage evaluation

    print(f"\n[QUERY] \n{query}")
    print(f"Using MMR retrieval k={k_value}. Then re-ranking.\n")

    for model_name, path in vectorstore_paths.items():
        print(f"--- Using model: {model_name} ---")

        # 1) Load embeddings & vectorstore
        embeddings = OllamaEmbeddings(model=model_name)
        vectorstore = FAISS.load_local(
            path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )

        # 2) MMR RETRIEVAL
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "filter": {"schema_type": schema},
                "k": k_value,
                "fetch_k": 100,
                "lambda_mult": 1,  # lower or raise to tweak MMR
            },
        )

        mmr_docs = retriever.invoke(query)
        print(f"Number of documents retrieved by MMR: {len(mmr_docs)}")

        # 2a) Print the top few from original MMR retrieval
        print(f"\n[MMR Top {top_n_to_print} Documents]:")
        for i, doc in enumerate(mmr_docs[:top_n_to_print], start=1):
            snippet = doc.page_content[:80].replace("\n", " ")
            print(f"  {i}. {snippet}...")

        # 3) SECOND PASS RERANK
        reranked_list = second_pass_rerank(query, mmr_docs, embeddings)

        # 3a) Print the top few from re-ranked results
        print(f"\n[Re-Ranked Top {top_n_to_print} Documents]:")
        for i, (doc, score) in enumerate(reranked_list[:top_n_to_print], start=1):
            snippet = doc.page_content[:80].replace("\n", " ")
            print(f"  {i}. Score={score:.4f} | {snippet}...")

        # 4) EVALUATION
        # We'll consider top_n_for_eval docs from re-ranked results as "final"
        final_top_docs = reranked_list[:top_n_for_eval]
        final_chunks = [doc.page_content for doc, _ in final_top_docs]
        evaluate_retrieval(model_name, final_chunks)

        print("-" * 70)


# -------------------------------------------------------------------------------------
# SECOND PASS RERANK FUNCTION
# -------------------------------------------------------------------------------------
def second_pass_rerank(
    query: str,
    docs: List[Document],
    embedding_function: OllamaEmbeddings
) -> List[Tuple[Document, float]]:
    """
    Compute a new score for each doc by re-embedding doc text & query,
    then do a dot product. Sort descending by score.
    """
    query_emb = embedding_function.embed_query(query)
    new_ranked_results = []
    for doc in docs:
        doc_emb = embedding_function.embed_query(doc.page_content)
        # Dot product
        score = np.dot(query_emb, doc_emb)
        new_ranked_results.append((doc, score))

    # Sort in descending order of score
    new_ranked_results.sort(key=lambda x: x[1], reverse=True)
    return new_ranked_results


# -------------------------------------------------------------------------------------
# EVALUATION: EXACT MATCH
# -------------------------------------------------------------------------------------
def evaluate_retrieval(model_name, retrieved_chunks):
    """
    Simple exact-match check against 'expected_chunks'.
    """
    print(f"\nExact-Match Evaluation for {model_name}:")
    correctly_retrieved = [c for c in expected_chunks if c in retrieved_chunks]
    missed_chunks = [c for c in expected_chunks if c not in retrieved_chunks]

    print(f"  Correctly retrieved chunks: {len(correctly_retrieved)}/{len(expected_chunks)}")
    for chunk in correctly_retrieved:
        print(f"    ✔ {chunk[:60]}...")

    print(f"  Missed chunks: {len(missed_chunks)}/{len(expected_chunks)}")
    for chunk in missed_chunks:
        print(f"    ✘ {chunk[:60]}...")


# -------------------------------------------------------------------------------------
# RUN EXAMPLE
# -------------------------------------------------------------------------------------
if __name__ == "__main__":
    schema = "EPD_DataSet"  # Example schema
    k_value = 80           # Adjust as desired

    # Adjust these paths to your local FAISS index
    embeddings_name = "row_cs3000_co0_faiss_index"
    vectorstore_paths = {
        "bge-m3:latest": f"../embeddings/bge-m3/{embeddings_name}",
        "snowflake-arctic-embed2:latest": f"../embeddings/snowflake/{embeddings_name}",
        # ... add more as needed
    }

    for schema_key, attributes in attribute_schema_mapping.items():
        two_stage_retrieval_and_eval(attributes, vectorstore_paths, schema_key, k_value)



[QUERY] 
UUID
Version
Name (de)
Name (en)
Kategorie (original)
Kategorie (en)
Konformität
Laenderkennung
Using MMR retrieval k=80. Then re-ranking.

--- Using model: bge-m3:latest ---
Number of documents retrieved by MMR: 80

[MMR Top 20 Documents]:
  1. 'Name des Konformitätssystems','Compliance system name','referenceToComplianceSy...
  2. 'Klassenname','Class name','class','nan','Name der Gliederungsklasse','~','Name ...
  3. 'Validierung','Validation','validation','nan','Validierungsinformationen','nan',...
  4. 'Registrierungsnummer','Registration number','registrationNumber','String','ID-N...
  5. 'Name','Name','epd2:name','StringMultiLang','Name des Materials/der Substanz.','...
  6. 'Eindeutiger Klassenidentifizierer','Unique class identifier','@classId','string...
  7. 'Vorhergehende Datensatzversion','Preceding data set version','referenceToPreced...
  8. 'Klassifizierungsinformation','Classification information','classificationInform...
  9. 'Konformität','Compliance','comp

In [148]:
import numpy as np
from typing import List, Tuple
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_ollama import OllamaEmbeddings


# -------------------------------------------------------------------------------------
# MAIN COMPARISON FUNCTION
# -------------------------------------------------------------------------------------
def two_stage_retrieval_and_eval(attributes, vectorstore_paths, schema, reranker_model, k_value=40):
    """
    1) MMR-based retrieval (k = k_value) using the primary model (e.g., bge-m3).
    2) Print the MMR retrieval.
    3) Re-rank those docs with a second pass using a different reranker_model.
    4) Print the re-ranked retrieval.
    5) Evaluate coverage of final top docs.
    """
    query = "\n".join(attributes)
    top_n_to_print = 20  # How many docs to show in each list for demonstration
    top_n_for_eval = 20  # How many docs to consider "final" for coverage evaluation

    print(f"\n[QUERY] {query}")
    print(f"Using MMR retrieval k={k_value}, then second-pass reranking with model='{reranker_model}'.\n")

    for model_name, path in vectorstore_paths.items():
        print(f"--- First-Pass Model: {model_name} ---")

        # 1) Load embeddings & vectorstore
        first_pass_embeddings = OllamaEmbeddings(model=model_name)
        vectorstore = FAISS.load_local(
            path,
            embeddings=first_pass_embeddings,
            allow_dangerous_deserialization=True
        )

        # 2) MMR RETRIEVAL (with the first-pass model)
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "filter": {"schema_type": schema},
                "k": k_value,
                "fetch_k": 100,
                "lambda_mult": 1,  # lower or raise to tweak MMR diversity
            },
        )

        mmr_docs = retriever.invoke(query)
        print(f"Number of documents retrieved by MMR: {len(mmr_docs)}")

        # 2a) Print the top few from original MMR retrieval
        print(f"\n[MMR Top {top_n_to_print} Documents]:")
        for i, doc in enumerate(mmr_docs[:top_n_to_print], start=1):
            snippet = doc.page_content[:80].replace("\n", " ")
            print(f"  {i}. {snippet}...")

        # 3) SECOND PASS RERANK (with the new reranker model)
        reranked_list = second_pass_rerank(query, mmr_docs, reranker_model)

        # 3a) Print the top few from re-ranked results
        print(f"\n[Re-Ranked Top {top_n_to_print} Documents using '{reranker_model}':]")
        for i, (doc, score) in enumerate(reranked_list[:top_n_to_print], start=1):
            snippet = doc.page_content[:80].replace("\n", " ")
            print(f"  {i}. Score={score:.4f} | {snippet}...")

        # 4) EVALUATION
        # We'll consider top_n_for_eval docs from re-ranked results as "final"
        final_top_docs = reranked_list[:top_n_for_eval]
        final_chunks = [doc.page_content for doc, _ in final_top_docs]
        evaluate_retrieval(model_name + " -> " + reranker_model, final_chunks)

        print("-" * 70)


# -------------------------------------------------------------------------------------
# SECOND PASS RERANK FUNCTION
# -------------------------------------------------------------------------------------
def second_pass_rerank(
    query: str,
    docs: List[Document],
    reranker_model: str
) -> List[Tuple[Document, float]]:
    """
    Compute a new score for each doc by re-embedding doc text & query with a
    different model ('reranker_model'), then do a dot product. Sort descending.
    """
    # Load the reranker embeddings
    reranker_embeddings = OllamaEmbeddings(model=reranker_model)

    # Embed the query with the reranker model
    query_emb = reranker_embeddings.embed_query(query)

    new_ranked_results = []
    for doc in docs:
        # Re-embed each doc's content with the reranker model
        doc_emb = reranker_embeddings.embed_query(doc.page_content)
        # Dot product
        score = np.dot(query_emb, doc_emb)
        new_ranked_results.append((doc, score))

    # Sort in descending order of score
    new_ranked_results.sort(key=lambda x: x[1], reverse=True)
    return new_ranked_results


# -------------------------------------------------------------------------------------
# EVALUATION: EXACT MATCH
# -------------------------------------------------------------------------------------
def evaluate_retrieval(model_name, retrieved_chunks):
    """
    Simple exact-match check against 'expected_chunks'.
    """
    print(f"\nExact-Match Evaluation for {model_name}:")
    correctly_retrieved = [c for c in expected_chunks if c in retrieved_chunks]
    missed_chunks = [c for c in expected_chunks if c not in retrieved_chunks]

    print(f"  Correctly retrieved chunks: {len(correctly_retrieved)}/{len(expected_chunks)}")
    for chunk in correctly_retrieved:
        print(f"    ✔ {chunk[:60]}...")

    print(f"  Missed chunks: {len(missed_chunks)}/{len(expected_chunks)}")
    for chunk in missed_chunks:
        print(f"    ✘ {chunk[:60]}...")


# -------------------------------------------------------------------------------------
# RUN EXAMPLE
# -------------------------------------------------------------------------------------
if __name__ == "__main__":
    # Example usage
    schema = "EPD_DataSet"
    k_value = 80  # broad retrieval

    # Suppose you have these vectorstore indexes:
    embeddings_name = "row_cs3000_co0_faiss_index"
    vectorstore_paths = {
        "bge-m3:latest": f"../embeddings/bge-m3/{embeddings_name}",
        "snowflake-arctic-embed2:latest": f"../embeddings/snowflake/{embeddings_name}",
        # add more if you like
    }

    # The new reranker model
    reranker_model = "linux6200/bge-reranker-v2-m3:latest"

    for schema_key, attributes in attribute_schema_mapping.items():
        two_stage_retrieval_and_eval(
            attributes=attributes,
            vectorstore_paths=vectorstore_paths,
            schema=schema_key,
            reranker_model=reranker_model,
            k_value=k_value,
        )



[QUERY] Find the most sematically close to these attributes:
UUID
Version
Name (de)
Name (en)
Kategorie (original)
Kategorie (en)
Konformität
Laenderkennung
Using MMR retrieval k=80, then second-pass reranking with model='linux6200/bge-reranker-v2-m3:latest'.

--- First-Pass Model: bge-m3:latest ---
Number of documents retrieved by MMR: 80

[MMR Top 20 Documents]:
  1. 'Name des Konformitätssystems','Compliance system name','referenceToComplianceSy...
  2. 'Validierung','Validation','validation','nan','Validierungsinformationen','nan',...
  3. 'Vorhergehende Datensatzversion','Preceding data set version','referenceToPreced...
  4. 'Name','Name','epd2:name','StringMultiLang','Name des Materials/der Substanz.','...
  5. 'Datensatzversion','Data set version','dataSetVersion','Version','Versionsnummer...
  6. 'UUID des Datensatzes','UUID of Process data set','UUID','UUID','UUID des Datens...
  7. 'Einheitengruppe','Unit group','epd:referenceToUnitGroupDataSet','GlobalReferenc...
  8. 'Ein

In [7]:
import numpy as np
from typing import List, Tuple
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


# -------------------------------------------------------------------------------------
# MAIN COMPARISON FUNCTION
# -------------------------------------------------------------------------------------
def two_stage_retrieval_and_eval(attributes, vectorstore_paths, schema, reranker_model, k_value=40):
    """
    1) MMR-based retrieval (k = k_value) using the primary model (e.g., bge-m3).
    2) Print the MMR retrieval.
    3) Re-rank those docs with a second pass using a different reranker_model.
    4) Print the re-ranked retrieval.
    5) Evaluate coverage of final top docs.
    """
    query = "\n".join(attributes)
    top_n_to_print = 20  # How many docs to show in each list for demonstration
    top_n_for_eval = 20  # How many docs to consider "final" for coverage evaluation

    print(f"\n[QUERY] {query}")
    print(f"Using MMR retrieval k={k_value}, then second-pass reranking with model='{reranker_model}'.\n")

    for model_name, path in vectorstore_paths.items():
        print(f"--- First-Pass Model: {model_name} ---")

        # 1) Load embeddings & vectorstore
        if model_name == "jinaai/jina-embeddings-v3":
            first_pass_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"trust_remote_code": True, "use_flash_attn": False}
    )
        else:
            first_pass_embeddings = OllamaEmbeddings(model=model_name)
        vectorstore = FAISS.load_local(
            path,
            embeddings=first_pass_embeddings,
            allow_dangerous_deserialization=True
        )

        # 2) MMR RETRIEVAL (with the first-pass model)
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "filter": {"schema_type": schema},
                "k": k_value,
                "fetch_k": 100,
                "lambda_mult": 1,  # lower or raise to tweak MMR diversity
            },
        )

        mmr_docs = retriever.invoke(query)
        print(f"Number of documents retrieved by MMR: {len(mmr_docs)}")

        # 2a) Print the top few from original MMR retrieval
        print(f"\n[MMR Top {top_n_to_print} Documents]:")
        for i, doc in enumerate(mmr_docs[:top_n_to_print], start=1):
            snippet = doc.page_content[:80].replace("\n", " ")
            print(f"  {i}. {snippet}...")

        # 3) SECOND PASS RERANK (with the new reranker model)
        reranked_list = second_pass_rerank(query, mmr_docs, reranker_model)

        # 3a) Print the top few from re-ranked results
        print(f"\n[Re-Ranked Top {top_n_to_print} Documents using '{reranker_model}':]")
        for i, (doc, score) in enumerate(reranked_list[:top_n_to_print], start=1):
            snippet = doc.page_content[:80].replace("\n", " ")
            print(f"  {i}. Score={score:.4f} | {snippet}...")

        # 4) EVALUATION
        # We'll consider top_n_for_eval docs from re-ranked results as "final"
        final_top_docs = reranked_list[:top_n_for_eval]
        final_chunks = [doc.page_content for doc, _ in final_top_docs]
        evaluate_retrieval(model_name + " -> " + reranker_model, final_chunks)

        print("-" * 70)


# -------------------------------------------------------------------------------------
# SECOND PASS RERANK FUNCTION
# -------------------------------------------------------------------------------------
def second_pass_rerank(
    query: str,
    docs: List[Document],
    reranker_model: str
) -> List[Tuple[Document, float]]:
    """
    Compute a new score for each doc by re-embedding doc text & query with a
    different model ('reranker_model'), then do a dot product. Sort descending.
    """
    # Load the reranker embeddings
    reranker_embeddings = HuggingFaceEmbeddings(
        model_name=reranker_model,
        model_kwargs={"trust_remote_code": True}
    )
    # Embed the query with the reranker model
    query_emb = reranker_embeddings.embed_query(query)

    new_ranked_results = []
    for doc in docs:
        # Re-embed each doc's content with the reranker model
        doc_emb = reranker_embeddings.embed_query(doc.page_content)
        # Dot product
        score = np.dot(query_emb, doc_emb)
        new_ranked_results.append((doc, score))

    # Sort in descending order of score
    new_ranked_results.sort(key=lambda x: x[1], reverse=True)
    return new_ranked_results


# -------------------------------------------------------------------------------------
# EVALUATION: EXACT MATCH
# -------------------------------------------------------------------------------------
def evaluate_retrieval(model_name, retrieved_chunks):
    """
    Simple exact-match check against 'expected_chunks'.
    """
    print(f"\nExact-Match Evaluation for {model_name}:")
    correctly_retrieved = [c for c in expected_chunks if c in retrieved_chunks]
    missed_chunks = [c for c in expected_chunks if c not in retrieved_chunks]

    print(f"  Correctly retrieved chunks: {len(correctly_retrieved)}/{len(expected_chunks)}")
    for chunk in correctly_retrieved:
        print(f"    ✔ {chunk[:60]}...")

    print(f"  Missed chunks: {len(missed_chunks)}/{len(expected_chunks)}")
    for chunk in missed_chunks:
        print(f"    ✘ {chunk[:60]}...")


# -------------------------------------------------------------------------------------
# RUN EXAMPLE
# -------------------------------------------------------------------------------------
if __name__ == "__main__":
    # Example usage
    schema = "EPD_DataSet"
    k_value = 80  # broad retrieval

    # Suppose you have these vectorstore indexes:
    embeddings_name = "row_cs3000_co0_faiss_index"
    vectorstore_paths = {
        # "bge-m3:latest": f"../embeddings/bge-m3/{embeddings_name}",
        # "snowflake-arctic-embed2:latest": f"../embeddings/snowflake/{embeddings_name}",
        "jinaai/jina-embeddings-v3": f"../embeddings/jinaai_jina-embeddings-v3/{embeddings_name}"
        # add more if you like
    }

    # The new reranker model
    # reranker_model = "linux6200/bge-reranker-v2-m3:latest"
    reranker_model = "jinaai/jina-reranker-v2-base-multilingual"

    for schema_key, attributes in attribute_schema_mapping.items():
        two_stage_retrieval_and_eval(
            attributes=attributes,
            vectorstore_paths=vectorstore_paths,
            schema=schema_key,
            reranker_model=reranker_model,
            k_value=k_value,
        )



[QUERY] UUID
Version
Name (de)
Name (en)
Kategorie (original)
Kategorie (en)
Konformität
Laenderkennung
Typ
Referenzjahr
Gueltig bis
URL
Declaration owner
Veroeffentlicht am
Registrierungsnummer
Registrierungsstelle
UUID des Vorgängers
Version des Vorgängers
URL des Vorgängers
Modul
Szenario
Szenariobeschreibung
Using MMR retrieval k=80, then second-pass reranking with model='jinaai/jina-reranker-v2-base-multilingual'.

--- First-Pass Model: jinaai/jina-embeddings-v3 ---


TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'use_flash_attn'